In [3]:
import numpy as np
import geopandas as gpd

In [4]:
########### USER ###########
#set domain
domain = 'WY'

In [5]:
# path to DEM ascii
DEMpath = '/nfs/attic/dfh/Aragon2/CSOdmn/'+domain+'/DEM_'+domain+'.asc'

#path to SNOTEL gdf
gdfpath = '/nfs/attic/dfh/Aragon2/CSOdata/'+domain+'/CSO_SNOTEL_sites.geojson'

# outfile path
outFpath = '/nfs/attic/dfh/Aragon2/CSOdmn/'+domain+'/snowmodel_line_pts.dat'

# path to SnowModel
modpath = '/nfs/attic/dfh/Aragon2/mar2020_snowmodel-dfhill/'

# first run the Fetch_SNOTEL notebook to find all of the stations within the modeling domain
# this will save lat lon coordinates of stations to projected cordinates that match the DEM

stn_gdf = gpd.read_file(gdfpath)

In [39]:
def getmeta(file_name):
    myvars = {}
    lines = open(file_name, 'r').readlines()
    for i in range(5):
        line=lines[i]
        name, var = line.partition("\t")[::2]
        myvars[name.strip()] = int(var)
    return myvars

In [40]:
myvars = getmeta(DEMpath)
myvars

{'ncols': 821,
 'nrows': 1231,
 'xllcorner': 570400,
 'yllcorner': 4832800,
 'cellsize': 100}

In [33]:
#Build northing easting array 
#Northing
# locate center of lower left cell
st = myvars['yllcorner']+(myvars['cellsize']/2)
#build northing array
north = np.flip(np.arange(st,st+myvars['nrows']*myvars['cellsize'],myvars['cellsize']),0)

#Easting
# locate center of lower left cell
st = myvars['xllcorner']+(myvars['cellsize']/2)
#build easting array
east = np.arange(st,st+myvars['ncols']*myvars['cellsize'],myvars['cellsize'])

In [34]:
#fortran indexing starts at 1
#staion 1 to N of N stations
count = np.zeros(stn_gdf.shape[0])
#easting of pixel corresponding to station 
stn_est = np.zeros(stn_gdf.shape[0])
#northing of pixel corresponding to station
stn_nor = np.zeros(stn_gdf.shape[0])
#index of pixel corresponding to station 
est_idx = np.zeros(stn_gdf.shape[0])
#index of pixel corresponding to station 
nor_idx = np.zeros(stn_gdf.shape[0])

for z in range(stn_gdf.shape[0]):
    count[z] = z + 1
    lons = abs(stn_gdf.easting[z]-east)
    loIDX = [i for i, value in enumerate(lons) if value == np.min(abs(stn_gdf.easting[z]-east))] 
    stn_est[z] = east[loIDX[0]]
    est_idx[z] = loIDX[0] + 1
    lats = abs(stn_gdf.northing[z]-north)
    laIDX = [i for i, value in enumerate(lats) if value == np.min(abs(stn_gdf.northing[z]-north))]
    stn_nor[z] = north[laIDX[0]]
    nor_idx[z] = laIDX[0] + 1

In [35]:
print(count)

[1. 2. 3. 4.]


In [36]:
#Print out .dat file 
#move this file to extra_met directory in SnowModel code
#https://pyformat.info/
f= open(outFpath,"w+")
for z in range(count.shape[0]):
    f.write('{:08.0f}\t'.format(count[z])+'{:10.0f}\t'.format(est_idx[z])+'{:10.0f}\t'.format(nor_idx[z])+
            '{:10.0f}\t'.format(stn_est[z])+'{:10.0f}\t\n'.format(stn_nor[z]))
f.close() 